# Make predictions

In [1]:
import pandas as pd
import re
from os import listdir
from multiprocessing import Pool
import unicodedata
from tqdm import tqdm
import numpy as np
from time import sleep

In [2]:
data = pd.read_pickle('/Users/yizhuoli/Downloads/WebCrawler/RICF_private/output_bin/df_proj.pkl.gz', compression='gzip')

In [3]:
data.head()

,proj_name,service_area,proj_desc_zh,proj_desc_en
0,购买导师、学位服,教育,支助武汉纺织大学教育事业发展,Support the development of education in Wuhan ...
1,奖励教师,教育,奖励教师,Reward teacher
2,奖励资助学生,教育,奖励资助学生,Reward funded students
3,教师及学生培训,教育,教师及学生培训费用,Teacher and student training fees
4,资助义务教育学生,教育,资助义务教育阶段,Funding compulsory education


In [4]:
# %load /Users/yizhuoli/Downloads/WebCrawler/npo_classifier/API/npoclass.py
################################### Define reproducibility ##########################
# # Force using CPU.
# # https://stackoverflow.com/questions/40690598/can-keras-with-tensorflow-backend-be-forced-to-use-cpu-or-gpu-at-will
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# obtain reproducible results

import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


################################### Import dependencies ##########################
from spellchecker import SpellChecker
from keras.preprocessing.sequence import pad_sequences
import pickle
from keras.models import load_model
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_list=[str.upper(s) for s in stopwords.words('english')+list(string.punctuation)]
import warnings
warnings.simplefilter('ignore')
import numpy as np
from keras.utils import np_utils
import os
import pandas as pd
import tensorflow as tf
from multiprocessing import Pool
p=Pool()


################################### Load saved models and classes ##########################
model_broad_cat=load_model('/Users/yizhuoli/Downloads/WebCrawler/npo_classifier/output/broad_category_model.h5')
model_major_group=load_model('/Users/yizhuoli/Downloads/WebCrawler/npo_classifier/output/major_group_model.h5')
with open('/Users/yizhuoli/Downloads/WebCrawler/npo_classifier/output/tokenizer.pkl', 'rb') as tokenizer_pkl:
    tokenizer = pickle.load(tokenizer_pkl)
with open('/Users/yizhuoli/Downloads/WebCrawler/npo_classifier/output/lb_broad_cat.pkl', 'rb') as lb_broad_cat_pkl:
    lb_broad_cat = pickle.load(lb_broad_cat_pkl)
with open('/Users/yizhuoli/Downloads/WebCrawler/npo_classifier/output/lb_major_group.pkl', 'rb') as lb_major_group_pkl:
    lb_major_group = pickle.load(lb_major_group_pkl)

# String/String list input --> a list of string token list(s) --> spellchecking (parallel) --> predict class (serial).


################################### Define functions ##########################
def npoclass(string_input=None):
    ## Define local function.
    # Spell check function. Return corrected word if unknown; return original word if known.
    def spellcheck(input_string):
        if type(input_string)==str:
            word_token_list=nltk.word_tokenize(input_string)
            return [s.upper() for s in p.map(SpellChecker().correction, word_token_list)]
        elif type(input_string)==list:
            word_token_list_list=[nltk.word_tokenize(string) for string in input_string]
            word_token_list_list_chk=[]
            for word_token_list in word_token_list_list:
                word_token_list_list_chk+=[[s.upper() for s in p.map(SpellChecker().correction, word_token_list)]]
            return word_token_list_list_chk
        else:
            raise NameError('Input must be a string or a list of strings.')
    
    result_dict={}
    # Text to sequences.
    # seq_encoding_text_test=tokenizer.texts_to_sequences(spellcheck(input_string_list))
    seq_encoding_text=tokenizer.texts_to_sequences(spellcheck(string_input))
    # Pads sequences to the same length (i.e., prepare matrix).
    x_text=pad_sequences(sequences=seq_encoding_text,
                        maxlen=46612, # Max length of the sequence.
                        dtype = "int32", padding = "post", truncating = "post", 
                        value = 0 # Zero is used for representing None or Unknown.
                         )
    # Predict.
    y_prob=model_major_group.predict(x_text)
    result_dict['major_group_label']=lb_major_group.inverse_transform(np_utils.to_categorical(y_prob.argmax(axis=-1))).tolist()
    result_dict['major_group_prob']=[s.max() for s in y_prob]
    y_prob=model_broad_cat.predict(x_text)
    result_dict['broad_category_label']=lb_broad_cat.inverse_transform(np_utils.to_categorical(y_prob.argmax(axis=-1))).tolist()
    result_dict['broad_category_prob']=[s.max() for s in y_prob]
    return result_dict

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yizhuoli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
len(list(data['proj_desc_en']))

10068

In [6]:
results = npoclass(list(data['proj_desc_en']))

In [8]:
results_df = pd.DataFrame.from_dict(results)
results_df.to_csv('chinese_pred.csv')

# Check the results

In [4]:
results_df = pd.read_csv('chinese_pred.csv')

In [5]:
results_df.sample(10)

,Unnamed: 0,major_group_label,major_group_prob,broad_category_label,broad_category_prob
9030,9030,N,0.001467,VII,0.050357
9229,9229,B,0.001157,II,0.087522
5968,5968,B,0.036328,II,0.067784
7231,7231,B,0.046549,II,0.040705
543,543,T,0.001950,V,0.006436
4601,4601,G,0.017430,VII,0.019226
463,463,A,0.020565,VII,0.055867
2313,2313,W,0.313761,VII,0.166105
8748,8748,B,0.015679,V,0.029133
8352,8352,S,0.001754,VII,0.045203


I. Arts, Culture, and Humanities - A  
II. Education - B  
III. Environment and Animals - C, D  
IV. Health - E, F, G, H  
V. Human Services - I, J, K, L, M, N, O, P  
VI. International, Foreign Affairs - Q  
VII. Public, Societal Benefit - R, S, T, U, V, W  
VIII. Religion Related - X  
IX. Mutual/Membership Benefit - Y  
X. Unknown, Unclassified - Z

In [6]:
broad_cat_dict={'I': ['文化艺术'],
                'II': ['教育'],
                'III': ['生态环境'],
                'IV': ['医疗卫生'],
                'V': ['志愿服务', '体育', '灾害救助'],
                'VII': ['扶贫及社区发展', '社会服务', '公益事业发展', '法律与公民权力', '政策倡导', '科学研究'],
               }

In [7]:
def zh_ntee2cat(string):
    global broad_cat_dict
    return [s for s in broad_cat_dict.keys() if string in broad_cat_dict[s]][0]

data['zh_broad_cat']=data['service_area'].apply(zh_ntee2cat)

In [37]:
# Pay attention to the tail pred, it's NaN here
data.tail(10)

,proj_name,service_area,proj_desc_zh,proj_desc_en,zh_broad_cat,pred
11035,浙江省文艺演出团赴印尼交流访问,文化艺术,"2月9日至17日,省对外友好交流基金会组织了由浙江歌舞剧院、浙江曲杂总团演职人员和杭州师范大...","From February 9th to 17th, the Provincial Fore...",I,NaN
11036,浙江省茶文化交流团赴日本访问,文化艺术,"4月4日至4月8日,省对外友好交流基金会组织了浙江省茶文化交流团一行15人访问日本,参加在静...","From April 4th to April 8th, the Provincial Fo...",I,NaN
11037,安养活动,社会服务,"2017年1月春节慰问社区贫困户五保户,共计慰问金额26600元2017年10月重阳节慰问老...","In the Spring Festival of January 2017, the fi...",VII,NaN
11038,扶贫救灾,扶贫及社区发展,"2017年8月欧江岔镇高平村遭受洪灾,重灾户3户,五保户3户每户发放救灾款2000元,余下4...","In August 2017, Gaoping Village in Oujiangyu T...",VII,NaN
11039,基金会助学,教育,"2017年8月,一对一帮扶捐助中小学生96名,每人2000元,并给优秀学生物质奖励3000元...","In August 2017, one-on-one support for 96 prim...",II,NaN
11040,大病救助,医疗卫生,"2017年肾病患儿肖麟每月获得基金会500元捐助,共计6000元1月份广法寺法师重病住院获基...","In 2017, Xiao Lin, a child with kidney disease...",IV,NaN
11041,村级卫生室捐款,扶贫及社区发展,2017年10月江华瑶族自治县河路口镇招礼村修建村级卫生室捐助5万元。,"In October 2017, Jiangli Yao Autonomous County...",VII,NaN
11045,低保家庭子女大学新生入学资助,教育,根据《诸暨市人民政府关于进一步完善帮困助学制度的实施意见》(诸政发[2003]101号)精神...,"According to the spirit of the ""Zhuji City Peo...",II,NaN
11046,薄弱乡镇教育经费补助,教育,"为努力推动全社会关心和支持教育,促进诸暨市教育事业的均衡、协调发展,本年度,用于薄弱乡镇学校...",In an effort to promote the care and support o...,II,NaN
11047,省级三等奖以上获奖者的奖励,教育,"根据诸教〔1991〕29号文件及诸暨市人民教育基金会发布的竞赛获奖者奖励办法的有关精神,经学...","According to the relevant spirit of the ""Natio...",II,NaN


In [34]:
print(len(data))
print(len(results_df))

10068
10068


Problem:
1. The length of data and results_df are same, however, the index of data is end at 11047. So, if we just use 
```
data['pred'] = results_df['broad_category_label']
```
the result will ust be 
```
print('Overall ACC:', len(data[data.pred==data.zh_broad_cat])/len(data))
>>>Overall ACC: 0.2679777512912197
```
Found this when checking the results by doing
```
data.groupby('pred').count()['proj_name']/len(data)
```
and 
```
results_df.groupby('broad_category_label').count()['Unnamed: 0']/len(results_df)
```
Then we found the result of these two are not same. So there must be something wrong. Then we check the 
```
data.tail()
data.pred[-100:]
data.pred[9100:9200]
```
Then we could find that because of the wrong index, the df doesn't match with each otehr.

In [24]:
results_df.tail()

,Unnamed: 0,major_group_label,major_group_prob,broad_category_label,broad_category_prob
10063,10063,A,0.001346,I,0.124221
10064,10064,W,0.001393,I,0.329924
10065,10065,B,0.065964,II,0.557183
10066,10066,B,0.002276,II,0.100837
10067,10067,B,0.172697,II,0.844221


In [40]:
data.reset_index(drop=True, inplace=True)

In [41]:
data.tail()

,proj_name,service_area,proj_desc_zh,proj_desc_en,zh_broad_cat,pred
10063,大病救助,医疗卫生,"2017年肾病患儿肖麟每月获得基金会500元捐助,共计6000元1月份广法寺法师重病住院获基...","In 2017, Xiao Lin, a child with kidney disease...",IV,NaN
10064,村级卫生室捐款,扶贫及社区发展,2017年10月江华瑶族自治县河路口镇招礼村修建村级卫生室捐助5万元。,"In October 2017, Jiangli Yao Autonomous County...",VII,NaN
10065,低保家庭子女大学新生入学资助,教育,根据《诸暨市人民政府关于进一步完善帮困助学制度的实施意见》(诸政发[2003]101号)精神...,"According to the spirit of the ""Zhuji City Peo...",II,NaN
10066,薄弱乡镇教育经费补助,教育,"为努力推动全社会关心和支持教育,促进诸暨市教育事业的均衡、协调发展,本年度,用于薄弱乡镇学校...",In an effort to promote the care and support o...,II,NaN
10067,省级三等奖以上获奖者的奖励,教育,"根据诸教〔1991〕29号文件及诸暨市人民教育基金会发布的竞赛获奖者奖励办法的有关精神,经学...","According to the relevant spirit of the ""Natio...",II,NaN


In [42]:
data['pred'] = results_df['broad_category_label']
print('Overall ACC:', len(data[data.pred==data.zh_broad_cat])/len(data))

Overall ACC: 0.5406237584425904


In [44]:
data.tail()

,proj_name,service_area,proj_desc_zh,proj_desc_en,zh_broad_cat,pred
10063,大病救助,医疗卫生,"2017年肾病患儿肖麟每月获得基金会500元捐助,共计6000元1月份广法寺法师重病住院获基...","In 2017, Xiao Lin, a child with kidney disease...",IV,I
10064,村级卫生室捐款,扶贫及社区发展,2017年10月江华瑶族自治县河路口镇招礼村修建村级卫生室捐助5万元。,"In October 2017, Jiangli Yao Autonomous County...",VII,I
10065,低保家庭子女大学新生入学资助,教育,根据《诸暨市人民政府关于进一步完善帮困助学制度的实施意见》(诸政发[2003]101号)精神...,"According to the spirit of the ""Zhuji City Peo...",II,II
10066,薄弱乡镇教育经费补助,教育,"为努力推动全社会关心和支持教育,促进诸暨市教育事业的均衡、协调发展,本年度,用于薄弱乡镇学校...",In an effort to promote the care and support o...,II,II
10067,省级三等奖以上获奖者的奖励,教育,"根据诸教〔1991〕29号文件及诸暨市人民教育基金会发布的竞赛获奖者奖励办法的有关精神,经学...","According to the relevant spirit of the ""Natio...",II,II


In [47]:
# See the composition by NTEE major groups.
print(data.groupby('zh_broad_cat').count()['proj_name']/len(data), '\n'*2, results_df.groupby('broad_category_label').count()['Unnamed: 0']/len(results_df))

zh_broad_cat
I      0.038439
II     0.539829
III    0.027910
IV     0.087505
V      0.031784
VII    0.274533
Name: proj_name, dtype: float64 

 broad_category_label
I       0.101212
II      0.400576
III     0.012416
IV      0.078069
IX      0.005463
V       0.185141
VI      0.015693
VII     0.191498
VIII    0.009932
Name: Unnamed: 0, dtype: float64


In [62]:
right_res = data[data.pred==data.zh_broad_cat]

In [66]:
results_pro = pd.DataFrame()
results_pro['pred_right'] = right_res.groupby('zh_broad_cat').count()['proj_name']
results_pro['trueth'] = data.groupby('zh_broad_cat').count()['proj_name']

In [68]:
results_pro['pred_right'] / results_pro['trueth']

zh_broad_cat
I      0.682171
II     0.668997
III    0.209964
IV     0.526674
V      0.481250
VII    0.313314
dtype: float64

# Check the reason
What I can got here is to see what the more frequent words in the original texts and the chinese translated results.

In [69]:
#initializing count vectorizer to examine text-based data
from sklearn.feature_extraction.text import CountVectorizer

#Setting n-gram range from 1-3 is about as complex as I want to get. 
#I also want words that appear in at least 5% of postings, to cut out sparsely 
#represented words that won't have helpful discriminating power
cvec = CountVectorizer(stop_words='english',ngram_range=(1, 3), min_df = .05)

In [70]:
#start by fitting on desc
cvec.fit(data["proj_desc_en"])

#transforming
desc_cvec = pd.DataFrame(cvec.transform(data["proj_desc_en"]).todense(),
                       columns=cvec.get_feature_names())

desc_cvec.reset_index(drop=True, inplace=True)

In [72]:
desc_cvec.tail()

,00,00 yuan,000,000 yuan,000 yuan person,000 yuan year,10,10 000,10 000 yuan,100,...,yuan 2017,yuan person,yuan support,yuan total,yuan used,yuan year,yunnan,zhang,zhejiang,zhejiang university
10063,0,0,10,10,0,0,3,3,3,2,...,0,0,0,0,0,0,0,0,0,0
10064,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10065,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10067,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
#appending y_train to city_train to allow for easier examination of city words 
#associated with above vs. below median salaries
word_observe = pd.concat([desc_cvec, data['zh_broad_cat']], axis=1)
word_observe.head()

,00,00 yuan,000,000 yuan,000 yuan person,000 yuan year,10,10 000,10 000 yuan,100,...,yuan person,yuan support,yuan total,yuan used,yuan year,yunnan,zhang,zhejiang,zhejiang university,zh_broad_cat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,II
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,II
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,II
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,II
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,II


In [81]:
for i in ['I', 'II', 'III', 'IV', 'V', 'VII']:
    word_count = word_observe[word_observe["zh_broad_cat"]==i].sum(axis=0)[:-1]
    print (i, "most common words")
    cw = word_count.sort_values(ascending = False).head(20)
    print (cw, '\n'*2)

I most common words
cultural          371
art               343
culture           326
chinese           277
2017              210
project           201
china             195
foundation        187
people            165
public            159
activities        144
traditional       143
development       137
exhibition        128
national          128
children          125
promote           122
research          120
welfare           114
public welfare    108
dtype: object 


II most common words
students       5726
school         5278
yuan           4272
education      3252
000            3076
project        2978
2017           2628
university     2614
000 yuan       2286
foundation     2101
teachers       1820
fund           1674
development    1668
donated        1634
scholarship    1445
year           1409
outstanding    1372
children       1356
poor           1335
support        1303
dtype: object 


III most common words
project                     443
protection                  423

In [83]:
import os

In [84]:
train_file_path='../../dataset/UCF/train/'
file_list=os.listdir(train_file_path)
df_train=pd.DataFrame()
for file in file_list:
    df_train=pd.concat([df_train, pd.read_pickle(train_file_path+file, compression='gzip')])

len(df_train)

154424

In [85]:
# Code as 10 broad categories.
broad_cat_dict={'I': ['A'],
                'II': ['B'],
                'III': ['C', 'D'],
                'IV': ['E', 'F', 'G', 'H'],
                'V': ['I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'],
                'VI': ['Q'],
                'VII': ['R', 'S', 'T', 'U', 'V', 'W'],
                'VIII': ['X'],
                'IX': ['Y'],
                'X': ['Z'],
               }
def ntee2cat(string):
    global broad_cat_dict
    return [s for s in broad_cat_dict.keys() if string in broad_cat_dict[s]][0]

df_train['mission_prgrm_spellchk']=df_train['TAXPAYER_NAME']+' '+df_train['mission_spellchk']+' '+df_train['prgrm_dsc_spellchk'] # Using spell-checked.
df_train['broad_cat']=df_train['NTEE1'].apply(ntee2cat)
print(len(df_train['mission_prgrm_spellchk']), len(df_train['NTEE1'].drop_duplicates()), len(df_train['broad_cat'].drop_duplicates()))

154424 25 9


In [86]:
df_train['mission_prgrm_spellchk'][:10]

1363470    MADISON AREA BUILDERS ASSOCIATION INC TO PROVI...
424991     PRASAD CHILDRENS DENTAL HEALTH PROGRAM INC TO ...
1816561    ROTARY CLUB OF MILTON-WINDWARD FOUNDATION INCO...
1558658    AMERICAN LEGION POST 241 PROVIDED SOCIAL RECRE...
1781284    ST MARYS HOSPITAL INC SAINT MARY IS HOSPITAL P...
2202860    NORTHERN ARIZONA VOLUNTEER MEDICAL AND SURGICA...
297296     BEL CANTO COMPANY INC TO PROVIDE CHORAL SINGIN...
394101     UNIVERSITY OF THE ANDES FOUNDATION INC THE PRI...
141481     CCBC PROPERTIES INC TO OWN , LEASE , AND MANAG...
1552925    SHAKEN BABY PREVENTION OF IDAHO INC TO BRING E...
Name: mission_prgrm_spellchk, dtype: object

In [87]:
eg_cvec = CountVectorizer(stop_words='english',ngram_range=(1, 3), min_df = .05)

In [88]:
eg_cvec.fit(df_train['mission_prgrm_spellchk'])

#transforming
desc_eg_cvec = pd.DataFrame(eg_cvec.transform(df_train['mission_prgrm_spellchk']).todense(),
                       columns=eg_cvec.get_feature_names())

desc_eg_cvec.reset_index(drop=True, inplace=True)

In [93]:
desc_eg_cvec.tail()

,2013,activities,american,annual,approximately,area,arts,assist,assistance,association,...,skills,social,state,students,support,training,various,work,year,youth
154419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
154420,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,1,0,1,0,0
154421,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
154422,2,3,0,0,1,0,0,0,0,0,...,1,2,0,0,0,0,0,2,1,0
154423,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
desc_eg_cvec.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [103]:
eg_word_observe = pd.concat([desc_eg_cvec, df_train['broad_cat']], axis=1)
eg_word_observe.head()

,2013,activities,american,annual,approximately,area,arts,assist,assistance,association,...,social,state,students,support,training,various,work,year,youth,broad_cat
0,0,0,0,2,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,VII
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,IV
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,VII
3,0,3,1,0,0,2,0,0,0,0,...,2,0,0,0,0,0,0,0,0,VII
4,0,0,3,0,1,1,0,0,0,1,...,1,6,0,1,3,0,0,6,0,IV


In [104]:
for i in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII']:
    eg_word_count = eg_word_observe[eg_word_observe["broad_cat"]==i].sum(axis=0)[:-1]
    print (i, "most common words")
    cw = eg_word_count.sort_values(ascending = False).head(20)
    print (cw, '\n'*2)

I most common words
arts            12751
community       11225
public           9074
programs         8812
education        7853
program          7234
students         6710
school           6455
provide          6371
educational      6118
new              5293
year             4988
children         4908
organization     4823
center           4718
promote          4623
local            4113
support          3973
events           3802
people           3493
dtype: object 


II most common words
school          41328
students        29155
education       20420
support         16742
provide         15382
programs        14009
program         13486
educational     13048
community       11784
college         11706
foundation      10833
high             9711
children         9532
services         9468
public           8915
high school      7664
activities       7402
scholarships     7165
organization     6608
members          6127
dtype: object 


III most common words
public          4581
ed

In [112]:
for i in ['I', 'II', 'III', 'IV', 'V', 'VII']:
    eg_word_count = eg_word_observe[eg_word_observe["broad_cat"]==i].sum(axis=0)[:-1]
    word_count = word_observe[word_observe["zh_broad_cat"]==i].sum(axis=0)[:-1]
    print (i, "most common words")
    eg_cw = eg_word_count.sort_values(ascending = False).head(20)
    cw = word_count.sort_values(ascending = False).head(20)
    for i in range(20):
        print (eg_cw.index[i], '     |       ', cw.index[i])
    print('\n'*2)

I most common words
arts      |        cultural
community      |        art
public      |        culture
programs      |        chinese
education      |        2017
program      |        project
students      |        china
school      |        foundation
provide      |        people
educational      |        public
new      |        activities
year      |        traditional
children      |        development
organization      |        exhibition
center      |        national
promote      |        children
local      |        promote
support      |        research
events      |        welfare
people      |        public welfare



II most common words
school      |        students
students      |        school
education      |        yuan
support      |        education
provide      |        000
programs      |        project
program      |        2017
educational      |        university
community      |        000 yuan
college      |        foundation
foundation      |        teacher

Problem I think:
1. There are some bad content like yuan, 000, year and 2017. These will influence the classifier.
2. Frequently used words in chinese and English description is not same.

Suggested sulution:
Build a new classifier for chinese content.